In [22]:
import tensorflow as tf
import pickle
import sys
import matplotlib.pyplot as plt1 
import numpy as np
from IPython.display import Audio, display
import time
from sklearn.model_selection import train_test_split
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D, concatenate, Conv1D,Conv2D, Flatten, Reshape, Embedding, GRU, SpatialDropout1D, LSTM, Dropout, BatchNormalization
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from itertools import permutations
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from scipy.stats import trim_mean

In [3]:
def done():
    framerate = 44100
    duration=0.6
    freq=300
    t = np.linspace(0,duration,framerate*duration)
    data = np.sin(2*np.pi*freq*t)
    display(Audio(data, rate=framerate, autoplay=True))

In [4]:
full = pd.read_csv("../data/drug_class_identification/all3.csv")
full = full.dropna()
full['atc'] = full['atc'].apply(lambda x : x[0])
full.head()

,smiles,name,id,inchi_key,atc
75,NC12CC3CC(CC(C3)C1)C2,amantadine,BRD-K70330367,DKNWSYNQZKUICI-UHFFFAOYSA-N,N
291,CS(=O)(=O)OCCCCOS(C)(=O)=O,busulfan,BRD-K23204545,COVZYZSDYWQREU-UHFFFAOYSA-N,L
322,CCCC(C)(COC(N)=O)COC(=O)NC(C)C,carisoprodol,BRD-A99939097,OFZCIYFFPZCNJE-UHFFFAOYSA-N,M
491,CCN(CC)C(=O)N1CCN(C)CC1,diethylcarbamazine,BRD-K45542189,RCKMWOKWVGPNJF-UHFFFAOYSA-N,P
510,CCN(CC)C(=S)SSC(=S)N(CC)CC,disulfiram,BRD-K32744045,AUZONCFQVSMFAP-UHFFFAOYSA-N,P


In [5]:
X = full["smiles"]
y = full['atc']

In [6]:
def getVocabulary(sample):
    vocabulary = set()
    for word in sample:
        for character in word:
            vocabulary.add(character)
    return (vocabulary)

In [7]:
characters = getVocabulary(X)
token_index = dict(zip(characters, range(1, len(characters) + 1)))

samples = X.tolist()
max_length = 70
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, character in enumerate(sample[:max_length]):
        index = token_index.get(character)
        results[i, j, index] = 1.

In [8]:
X = np.asarray(results)
X.shape

(1004, 70, 31)

In [12]:
# Define our own plot function
def scatter(x, y, subtitle=None):
    le = LabelEncoder()
    labels = le.fit_transform(y)

    # We choose a color palette with seaborn.
    palette = np.array(sns.color_palette("hls", 14))

    # We create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40,
                    c=palette[labels.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # We add the labels for each digit.
    txts = []
    for i in range(14):
        # Position of each label.
        xtext, ytext = trim_mean(x[labels == i, :], axis=0, proportiontocut=0.2)
        letter = le.inverse_transform([i])[0]
        txt = ax.text(xtext, ytext, str(letter), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)
        
    if subtitle != None:
        plt.suptitle(subtitle)
        
    plt.savefig(subtitle)

In [13]:
x_train, x_test,y_train,y_test = train_test_split(X,y)

x_train_flat = x_train.reshape(-1,70*31)
x_test_flat = x_test.reshape(-1,70*31)

tsne = TSNE()
train_tsne_embeds = tsne.fit_transform(x_train_flat[:512])
scatter(train_tsne_embeds, y_train[:512], "Samples from Training Data")

eval_tsne_embeds = tsne.fit_transform(x_test_flat[:512])
scatter(eval_tsne_embeds, y_test[:512], "Samples from Validation Data")

/Users/dweepa/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:2831: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)
/Users/dweepa/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:2831: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)


In [14]:
def triplet_loss(y_true, y_pred, alpha = 0.4):
    """
    Implementation of the triplet loss function
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor data
            positive -- the encodings for the positive data (similar to anchor)
            negative -- the encodings for the negative data (different from anchor)
    Returns:
    loss -- real number, value of the loss
    """
    print('y_pred.shape = ',y_pred)
    
    total_lenght = y_pred.shape.as_list()[-1]
#     print('total_lenght=',  total_lenght)
#     total_lenght =12
    
    anchor = y_pred[:,0:int(total_lenght*1/3)]
    positive = y_pred[:,int(total_lenght*1/3):int(total_lenght*2/3)]
    negative = y_pred[:,int(total_lenght*2/3):int(total_lenght*3/3)]

    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)

    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)

    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
 
    return loss

def baseNetwork():    
        model = Sequential()
        model.add(Reshape((70, 31), input_shape=(1004,None, None)))
        model.add(Conv1D(20,10,activation='relu'))
        model.add(BatchNormalization())
        model.add(Conv1D(20,5,activation='relu'))
        model.add(Conv1D(20,3,activation='relu'))
        model.add(Flatten())
        model.add(Dense(30, activation='relu'))
        model.add(Dropout(0.4))
        model.add(Dense(14, activation='softmax'))
        return model
    
anchor_input = Input((70,31,1, ), name='anchor_input')
positive_input = Input((70,31,1, ), name='positive_input')
negative_input = Input((70,31,1, ), name='negative_input')

# Shared embedding layer for positive and negative items
Shared_DNN = baseNetwork()


encoded_anchor = Shared_DNN(anchor_input)
encoded_positive = Shared_DNN(positive_input)
encoded_negative = Shared_DNN(negative_input)


merged_vector = concatenate([encoded_anchor, encoded_positive, encoded_negative], axis=-1, name='merged_layer')

model = Model(inputs=[anchor_input,positive_input, negative_input], outputs=merged_vector)
model.compile(loss=triplet_loss, optimizer='adam')
model.summary()

y_pred.shape =  Tensor("merged_layer/concat:0", shape=(?, 42), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 70, 31, 1)    0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 70, 31, 1)    0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 70, 31, 1)    0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 14)           43004       anchor_input[0][0]               
                               

In [15]:
def generateTriplet(x,y,testsize=0.2,ap_pairs=10,an_pairs=10):
    data_xy = tuple([x,y])

    trainsize = 1-testsize

    triplet_train_pairs = []
    triplet_test_pairs = []
    for data_class in sorted(set(data_xy[1])):

        same_class_idx = np.where((data_xy[1] == data_class))[0]
        diff_class_idx = np.where(data_xy[1] != data_class)[0]
        A_P_pairs = random.sample(list(permutations(same_class_idx,2)),k=ap_pairs) #Generating Anchor-Positive pairs
        Neg_idx = random.sample(list(diff_class_idx),k=an_pairs)
        

        #train
        A_P_len = len(A_P_pairs)
        Neg_len = len(Neg_idx)
        for ap in A_P_pairs[:int(A_P_len*trainsize)]:
            Anchor = data_xy[0][ap[0]]
            Positive = data_xy[0][ap[1]]
            for n in Neg_idx:
                Negative = data_xy[0][n]
                triplet_train_pairs.append([Anchor,Positive,Negative])               
        #test
        for ap in A_P_pairs[int(A_P_len*trainsize):]:
            Anchor = data_xy[0][ap[0]]
            Positive = data_xy[0][ap[1]]
            for n in Neg_idx:
                Negative = data_xy[0][n]
                triplet_test_pairs.append([Anchor,Positive,Negative])    
                
    return np.array(triplet_train_pairs), np.array(triplet_test_pairs)

In [16]:
X_train, X_test = generateTriplet(X,y)

In [17]:
Anchor = X_train[:,0,:].reshape(-1,70,31,1)
Positive = X_train[:,1,:].reshape(-1,70,31,1)
Negative = X_train[:,2,:].reshape(-1,70,31,1)
Anchor_test = X_test[:,0,:].reshape(-1,70,31,1)
Positive_test = X_test[:,1,:].reshape(-1,70,31,1)
Negative_test = X_test[:,2,:].reshape(-1,70,31,1)

Y_dummy = np.empty((Anchor.shape[0],300))
Y_dummy2 = np.empty((Anchor_test.shape[0],1))

model.fit([Anchor,Positive,Negative],y=Y_dummy,validation_data=([Anchor_test,Positive_test,Negative_test],Y_dummy2), epochs=50)

Train on 1120 samples, validate on 280 samples
Epoch 1/50
1120/1120 [==============================] - 3s 2ms/step - loss: 0.3516 - val_loss: 0.3161
Epoch 2/50
1120/1120 [==============================] - 1s 1ms/step - loss: 0.2474 - val_loss: 0.2687
Epoch 3/50
1120/1120 [==============================] - 1s 1ms/step - loss: 0.1883 - val_loss: 0.2377
Epoch 4/50
1120/1120 [==============================] - 1s 1ms/step - loss: 0.1396 - val_loss: 0.2257
Epoch 5/50
1120/1120 [==============================] - 1s 1ms/step - loss: 0.1131 - val_loss: 0.2543
Epoch 6/50
1120/1120 [==============================] - 1s 1ms/step - loss: 0.0965 - val_loss: 0.2343
Epoch 7/50
1120/1120 [==============================] - 1s 1ms/step - loss: 0.0833 - val_loss: 0.2153
Epoch 8/50
1120/1120 [==============================] - 1s 1ms/step - loss: 0.0761 - val_loss: 0.2300
Epoch 9/50
1120/1120 [==============================] - 1s 1ms/step - loss: 0.0666 - val_loss: 0.2336
Epoch 10/50
1120/1120 [============

In [18]:
trained_model = Model(inputs=anchor_input, outputs=encoded_anchor)

In [19]:
tsne = TSNE()
X_train_trm = trained_model.predict(x_train[:512].reshape(-1,70,31,1))
X_test_trm = trained_model.predict(x_test[:512].reshape(-1,70,31,1))
train_tsne_embeds = tsne.fit_transform(X_train_trm)
eval_tsne_embeds = tsne.fit_transform(X_test_trm)

In [20]:
scatter(train_tsne_embeds, y_train[:512], "Training Data After TNN")
scatter(eval_tsne_embeds, y_test[:512], "Validation Data After TNN")

/Users/dweepa/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:2831: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.mean(atmp[sl], axis=axis)


In [44]:
X_train_trm = trained_model.predict(x_train.reshape(-1,70,31,1))
X_test_trm = trained_model.predict(x_test.reshape(-1,70,31,1))

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

Classifier_input = Input((14,))
Classifier_output = Dense(14, activation='softmax')(Classifier_input)
Classifier_model = Model(Classifier_input, Classifier_output)


Classifier_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Classifier_model.fit(X_train_trm,y_train, validation_data=(X_test_trm,y_test),epochs=50)

Train on 753 samples, validate on 251 samples
Epoch 1/50
753/753 [==============================] - 1s 945us/step - loss: 2.6527 - acc: 0.0465 - val_loss: 2.6523 - val_acc: 0.0558
Epoch 2/50
753/753 [==============================] - 0s 69us/step - loss: 2.6351 - acc: 0.0850 - val_loss: 2.6381 - val_acc: 0.0677
Epoch 3/50
753/753 [==============================] - 0s 75us/step - loss: 2.6189 - acc: 0.0969 - val_loss: 2.6246 - val_acc: 0.0717
Epoch 4/50
753/753 [==============================] - 0s 67us/step - loss: 2.6036 - acc: 0.1116 - val_loss: 2.6116 - val_acc: 0.0757
Epoch 5/50
753/753 [==============================] - 0s 56us/step - loss: 2.5892 - acc: 0.1062 - val_loss: 2.5998 - val_acc: 0.0757
Epoch 6/50
753/753 [==============================] - 0s 62us/step - loss: 2.5756 - acc: 0.1102 - val_loss: 2.5888 - val_acc: 0.0757
Epoch 7/50
753/753 [==============================] - 0s 98us/step - loss: 2.5632 - acc: 0.1169 - val_loss: 2.5778 - val_acc: 0.0717
Epoch 8/50
753/753 [==

In [45]:
Classifier_model.evaluate(X_train_trm,y_train), Classifier_model.score(X_test_trm,y_test)

753/753 [==============================] - 0s 35us/step


AttributeError: 'Model' object has no attribute 'score'